In [4]:
!pip install --upgrade tensorflow

In [5]:
import numpy as np
np.random.seed(1337)
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from keras.utils import np_utils
import cv2 as cv
import pickle
import matplotlib.pyplot as plt



In [6]:
#loading the data we previously gathered

pickle_in = open("X_train.pickle","rb")
X_train = pickle.load(pickle_in)

pickle_in = open("y_train.pickle","rb")
y_train = pickle.load(pickle_in)

X_train = X_train/255.0


n_classes = 3
# print("Shape before one-hot encoding: ", y_train)
Y_train = np_utils.to_categorical(y_train, n_classes)
# print("Shape after one-hot encoding: ", Y_train)

print(len(y_train))
print(len(X_train))

model = Sequential()

model.add(Conv2D(filters = 8, kernel_size = (3, 3), activation = 'relu', padding = 'same', input_shape = (192, 192, 3)))
model.add(MaxPooling2D(pool_size=(2, 2), strides = 2))#1


model.add(Conv2D(filters = 16, kernel_size = (3, 3), activation = 'relu', padding = 'same', input_shape = (192, 192, 3)))
model.add(MaxPooling2D(pool_size=(2, 2), strides = 2))#2

model.add(tf.keras.layers.BatchNormalization())

model.add(Conv2D(filters = 32, kernel_size = (3, 3), activation = 'relu', padding = 'same', input_shape = (192, 192, 3)))
model.add(MaxPooling2D(pool_size=(2, 2), strides = 2))#3

model.add(tf.keras.layers.BatchNormalization())

model.add(Conv2D(filters = 64, kernel_size = (3, 3), activation = 'relu', padding = 'same', input_shape = (192, 192, 3)))
model.add(MaxPooling2D(pool_size=(2, 2), strides = 2))#4

model.add(Flatten())

model.add(Dense(units = n_classes, activation = 'softmax'))

model.add(Dense(3))
model.add(Activation('sigmoid'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.fit(X_train, Y_train, batch_size=32, epochs=10, validation_split=0.2)


#Saving the model on my computer
model.save('jump_rope_model.h5', overwrite=True)



3149
3149
Epoch 1/10
79/79 [==============================] - 46s 576ms/step - loss: 0.9803 - accuracy: 0.6435 - val_loss: 0.9988 - val_accuracy: 0.6317
Epoch 2/10
79/79 [==============================] - 56s 714ms/step - loss: 0.9148 - accuracy: 0.6526 - val_loss: 0.9797 - val_accuracy: 0.5508
Epoch 3/10
79/79 [==============================] - 51s 652ms/step - loss: 0.8476 - accuracy: 0.6511 - val_loss: 0.8249 - val_accuracy: 0.6571
Epoch 4/10
79/79 [==============================] - 56s 709ms/step - loss: 0.7132 - accuracy: 0.8174 - val_loss: 0.6415 - val_accuracy: 0.9857
Epoch 5/10
79/79 [==============================] - 53s 678ms/step - loss: 0.6263 - accuracy: 0.9762 - val_loss: 0.5644 - val_accuracy: 0.9889
Epoch 6/10
79/79 [==============================] - 54s 681ms/step - loss: 0.5484 - accuracy: 0.9853 - val_loss: 0.5333 - val_accuracy: 0.9556
Epoch 7/10
79/79 [==============================] - 56s 714ms/step - loss: 0.4879 - accuracy: 0.9893 - val_loss: 0.4552 - val_accura

In [7]:
#loading the previously saved model


model = tf.keras.models.load_model('jump_rope_model.h5')


def count_repetitions(video_path):
    #capturing the video so that we can
    #process it frame by frame
    capture = cv.VideoCapture(video_path)
    framerate = int(capture.get(5))
    resolution = (int(capture.get(3)),int(capture.get(4)))
    new_path = video_path[:-4] + '_with_repetitions_final.mp4'
    fourcc = cv.VideoWriter_fourcc(*"mp4v")
    #activating the 'VideoWriter' function so that 
    #we can put a live 'Jump Counter' above the video
    updated_frame = cv.VideoWriter(new_path,fourcc, framerate, resolution)
    repetitions = 0
    up = 0
    down = 0
    land = 0
    down_counter = 0
    up_counter = 0
    last_move = 'none'
    counter = 1
    ret, old_frame = capture.read()
    flow = np.zeros((old_frame.shape[0], old_frame.shape[1], 2), dtype=np.float32)
    params4farne = {'pyr_scale':0.5,'levels':3, 'winsize':15,'iterations': 3, 'poly_n': 5,'poly_sigma':1.2,'flags':cv.OPTFLOW_USE_INITIAL_FLOW}
    hsv = np.zeros_like(old_frame)
    hsv[...,1] = 255
    first_gray = cv.cvtColor(old_frame,cv.COLOR_BGR2GRAY)
    while(1):
        ret1,current_frame = capture.read()
        if ret1:
            second_gray = cv.cvtColor(current_frame,cv.COLOR_BGR2GRAY)
            flow = cv.calcOpticalFlowFarneback(first_gray,second_gray,None,0.5, 3, 15, 3, 5, 1.2, 0)
            mag,angle = cv.cartToPolar(flow[..., 0], flow[..., 1])
            hsv[...,0] = angle * 180 / np.pi/2
            hsv[...,2] = cv.normalize(mag,None,0,255,cv.NORM_MINMAX)
            RGB = cv.cvtColor(hsv,cv.COLOR_HSV2BGR)
            dims = (400,400)
            resized_frame_colored = cv.resize(RGB,dims,interpolation=cv.INTER_AREA)
            resized_frame_colored = cv.resize(resized_frame_colored, (192, 192))
            resized_frame_colored = resized_frame_colored.reshape((1,) + resized_frame_colored.shape)
            resized_frame_colored = resized_frame_colored/255.0
            pre = model.predict(resized_frame_colored)
            prediction = np.argmax(pre, axis=-1)[0]
            #Creating the Counter to determine how many jumps the user has performed
            #A jump is considered to be when a user goes up, down, and lands. 
            if prediction == 0:
                #The model sometimes has trouble distinguishing a downward movement
                #from a landing movement, to accomodate for this, I have the counter go up
                #if a user goes from down -> up without any 'landing' movement
                if down_counter > 3:
                    repetitions += 1
                if last_move == 'land' or last_move == 'down':
                    up_counter = 1
                else:
                    up_counter += 1
                down_counter = 0
                last_move = 'up'
            elif prediction == 1:
                last_move = 'down'
                down_counter += 1
                if down_counter > 7.5:
                    up_counter = 0
            elif prediction == 2:
                down_counter = 0
                if last_move == 'down':
                    if up_counter>1:
                        repetitions += 1
                last_move = 'land'
                up_counter = 0
            #Adding text to the video to show how many repetitions are being performed
            #in real time
            overlay_text = 'Jumps : {}'.format(repetitions)
            font = cv.FONT_HERSHEY_SIMPLEX
            fontscale = 2
            fontColor = (255,255,255)
            textloc = (350,250)
            lineType = 2
            cv.putText(current_frame, overlay_text,textloc,font, fontscale,fontColor,lineType)
            updated_frame.write(current_frame)
            counter += 1
            first_gray = second_gray
        else:
            #once the video is done processing
            #the function also returns the total amount of 
            #jumps a user has performed
            print('done')
            print(repetitions)
            break

In [8]:
count_repetitions("C:\\Users\\Dani\\Downloads\\testvid_original_2.mp4")

done
107
